### Kaggle Predicting Red Hat Business Value

As this my first kernel, I would very much appreciate some feedback.  While i'm confident that most of my process, particularly the initial clearning and transformation from cateogrical to numeric are correct, i'm more unsure about the steps I took in re: to train, test, split, etc.  

Thank you to everyone who contributes with tips and kernels of their own!

In [ ]:
# importing libraries

import numpy as np
import pandas as pd
from IPython.display import display, HTML

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# reading in data

people = pd.read_csv('../input/people.csv')
activity_train = pd.read_csv('../input/act_train.csv')
activity_test = pd.read_csv('../input/act_test.csv')

In [ ]:
# merging the dataframes into train, test

df = activity_train.merge(people, how='left', on='people_id' )
df_test = activity_test.merge(people, how='left', on='people_id' )

In [ ]:
# the shape of the dataframes

print (df.shape)
print (df_test.shape)

In [ ]:
# filling NaN values first

df = df.fillna('0', axis=0)
df_test = df_test.fillna('0', axis=0)

In [ ]:
# taking a look at the first few rows

df.head()

In [ ]:
df_test.head()

### preprocessing

In [ ]:
# a multi-column LabelEncoder()

# this solution for applying LabelEncoder() across multiple columns was suggested in the following thread
# http://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn

# I like this solution but is it the most efficient?  Would another method be more practical, particularly if 
# applied to different type of model 

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns 

    def fit(self,X,y=None):
        return self

    def transform(self,X):
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:
# defining a processor 

def processor(data):
    data = MultiColumnLabelEncoder(columns = ['people_id','activity_id', 'activity_category', 'date_x', 'char_1_x', 'char_2_x',
                                        'char_3_x', 'char_4_x', 'char_5_x', 'char_6_x', 'char_7_x', 'char_8_x', 'char_9_x',
                                        'char_10_x', 'char_1_y', 'group_1', 'char_2_y', 'date_y', 'char_3_y', 'char_4_y',
                                        'char_5_y', 'char_6_y', 'char_7_y', 'char_8_y', 'char_9_y']).fit_transform(df)
    
    bool_map = {True:1, False:0}

    data = data.applymap(lambda x: bool_map.get(x,x))
    
    return data

In [ ]:
# applying processor to training data

df_encoded = processor(df)

In [ ]:
df_encoded.head()

In [ ]:
df_encoded.dtypes

In [ ]:
# applying processor to test data

df_test_encoded = processor(df_test)

In [ ]:
df_test_encoded.head()

In [ ]:
df_test_encoded.dtypes

## modeling


In [ ]:
# defining X and y (features and target label)

X = df_encoded
y = X.pop('outcome')

In [ ]:
# shape of X and y

print (X.shape)
print (y.shape)

In [ ]:
'''

train, test, split the data.  hold out 25% for test

generally if not provided a test set, this would be the way to move forward.
yet I feel something is off in my process and would love feedback!

'''

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7)

In [ ]:
# random forest classifier

model = RandomForestClassifier(77, n_jobs=-1, random_state=7)
model.fit(X_train, y_train)
print ("model score ", model.score(X_test, y_test))

In [ ]:
# predicting test data

pred = model.predict(X_test)
pred

### Final thoughts

While the X_test a split of our traning data, as achieved through train, test, split, it is not the same as the actual test set provided in the data by kaggle.  For submission, this would be an issue.  For me, this competition is unique in although fairly straightforward, I haven't had to label encode categoricals on a provided test dataset.  Generally these are untouched until model prediction time.

Hopefully some other less experienced users such as myself can make some use of my inital sets in this process.